In [2]:
!pip install langchain-google-genai langchain-core langgraph pydantic google-generativeai --quiet
!pip install -q -U google-genai python-dotenv google-generativeai

In [27]:
# Setting up LLM
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types

load_dotenv()

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

def llm_call(prompt: str) -> str:
    response = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents=prompt,
        config=types.GenerateContentConfig(
            # It forces the model to output raw JSON only
            response_mime_type="application/json" 
        )
    )
    return response.text

# if __name__ == "__main__":
#     print(llm_call("Hello, Gemini!"))

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [12]:
# Setting up Gmail
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

# 1. SETUP GMAIL AUTHENTICATION
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    creds = None
    if os.path.exists('contents/token.json'):
        creds = Credentials.from_authorized_user_file('contents/token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('contents/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0, open_browser=False) # Modified to not open browser
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)

service = get_gmail_service()

# Check if gmail is connected
profile = service.users().getProfile(userId='me').execute()
print("Email:", profile["emailAddress"])

Email: forsignins043@gmail.com


In [19]:
def fetch_recent_emails(service, max_results=5):
    results = service.users().messages().list(
        userId="me",
        maxResults=max_results
    ).execute()

    messages = results.get("messages", [])
    full_email_objects = []

    for msg in messages:
        message = service.users().messages().get(
            userId="me",
            id=msg["id"],
            format="full"
        ).execute()
        
        full_email_objects.append(message) 

    return full_email_objects

In [14]:
import base64

def extract_body(payload: dict) -> str:
    body = ""

    if "data" in payload.get("body", {}):
        body = payload["body"]["data"]

    elif "parts" in payload:
        for part in payload["parts"]:
            if part.get("mimeType") == "text/plain" and "data" in part.get("body", {}):
                body = part["body"]["data"]
                break

    if body:
        return base64.urlsafe_b64decode(body).decode("utf-8", errors="ignore")

    return ""


def extract_email_parts(message: dict):
    payload = message.get("payload", {})
    headers = payload.get("headers", [])

    subject = ""
    sender = ""

    for h in headers:
        name = h.get("name", "").lower()
        if name == "subject":
            subject = h.get("value", "")
        elif name == "from":
            sender = h.get("value", "")

    body = extract_body(payload)

    return sender, subject, body


In [15]:
def email_for_llm(sender: str, subject: str, body: str) -> str:
    email_text = f"""
Sender: {sender}
Subject: {subject}

Body:
{body}
""".strip()
    
    return email_text

In [30]:
def traige_email(email_text: str) -> str:
    prompt = f"""
You are an expert AI Executive Assistant.
Your goal is to triage incoming emails into specific category

### CATEGORY DEFINITIONS:
1. **IGNORE**: Automated newsletters, marketing, receipts, system logs, or spam.
2. **NOTIFY**: informational emails where NO reply is expected (e.g., shipping updates, "FYI" memos, broad company announcements).
3. **RESPOND**: Emails that require a reply or action. This INCLUDES:
   - Invitations (Parties, Weddings, Meetings) that need an RSVP.**
   - Direct questions asked to Sanjay.
   - Scheduling requests.
   - Personal messages requiring acknowledgement.


Email:
{email_text}

Only return the category.
"""
    return llm_call(prompt)

In [23]:
import base64
from email.mime.text import MIMEText
import json

def create_draft_reply(service, sender: str, subject: str, body: str) -> str:
    current_feedback = ""
    
    while True:
        prompt = f"""
        You are a professional Email Assistant for Sanjay Sanapala.
        Your goal is to write a professional email reply.

        ### INPUT DATA:
        - Sender: {sender}
        - Original Subject: {subject}
        - Original Body: {body}
        
        ### USER FEEDBACK / ADJUSTMENTS:
        {current_feedback if current_feedback else "None (Draft the initial reply)"}

        ### GUIDELINES:
        1. **Tone:** Professional, direct, and polite.
        2. **Structure:** Greeting -> Main Point -> Call to Action -> Sign off.
        3. **Constraint:** Return strictly valid JSON.

        ### OUTPUT FORMAT (JSON ONLY):
        {{
            "To": "{sender}",
            "Subject": "Re: {subject} (or a better subject)",
            "Body": "The full email body text here..."
        }}
        """

        print("\n--- Generating Draft... ---")
        
        try:
            response_json = llm_call(prompt)
            reply_data = json.loads(response_json)
        except json.JSONDecodeError:
            print("Error: LLM did not return valid JSON. Retrying...")
            continue

        print(f"\nDRAFT PREVIEW:\nTo: {reply_data['To']}\nSubject: {reply_data['Subject']}\nBody:\n{reply_data['Body']}\n")
        
        user_choice = input("Action (yes / no / [type feedback]): ").strip().lower()

        if user_choice in ["yes", "y"]:
            # create_gmail_draft(service, reply_data["To"], reply_data["Subject"], reply_data["Body"])
            return "Success: Draft Created."    
        elif user_choice in ["no", "n"]:
            print("Operation cancelled.")
            return "Cancelled."
        else:
            print("Refining draft based on feedback...")
            current_feedback += f"\n- User requested change: {user_choice}"

In [32]:
if __name__ == "__main__":
    messages = fetch_recent_emails(service, 1)

    for msg in messages:
        sender, subject, body = extract_email_parts(msg)

        mail_text = email_for_llm(sender, subject, body)

        decision = traige_email(mail_text).strip()
        decision = decision.replace('"', '').replace("'", "").strip()
        
        print(f"Decision: {decision}")

        if decision == "IGNORE":
            print("Mail Ignored")
            
        elif decision == "NOTIFY":
            print(f"Notification: {subject}")
            
        elif decision == "RESPOND":
            create_draft_reply(service, sender, subject, body)

Decision: RESPOND

--- Generating Draft... ---

DRAFT PREVIEW:
To: Bluey Face <blueyface043@gmail.com>
Subject: Re: Invitation to the party
Body:
Dear Bluey,

Thank you very much for your kind invitation to the party on January 28th at 7:00 PM at your house.

I would be delighted to attend and look forward to celebrating with you.

Best regards,

Sanjay Sanapala

Refining draft based on feedback...

--- Generating Draft... ---

DRAFT PREVIEW:
To: Bluey Face <blueyface043@gmail.com>
Subject: Re: Invitation to the party
Body:
Dear Bluey,

Thank you for your kind invitation to the party on January 28th.

Unfortunately, I will be unable to attend due to a prior commitment.

I appreciate you thinking of me and I hope you have a wonderful time.

Best regards,

Sanjay Sanapala

